# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели.
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:
    - нужно будет добавить еще один тэг <start>
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так).

In [ ]:
!pip install razdel

In [ ]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML

In [ ]:
from collections import Counter

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
from scipy.sparse import lil_matrix

In [ ]:
news = open('lenta.txt', encoding='utf-8').read()

In [ ]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [ ]:
sentences_news = [['<start>','<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [ ]:
test_news = sentences_news[:50]
train_news = sentences_news[50:]

In [ ]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [ ]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in train_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence, n=2))
    trigrams_news.update(ngrammer(sentence, n=3))

In [ ]:
# матрица слова на биграммы (инициализируем нулями)
matrix_news = lil_matrix((len(bigrams_news),
                         len(unigrams_news)))

# к матрице нужно обращаться по индексам
# поэтому зафиксируем порядок слов в словаре и сделаем маппинг id-слово и слово-id
id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

# то же самое для биграмм
id2bigram_news = list(bigrams_news)
bigram2id_news = {word:i for i, word in enumerate(id2bigram_news)}

In [ ]:
# заполняем матрицу
for trigram in trigrams_news:
    word1, word2, word3 = trigram.split()
    bigram = word1+' '+word2
    # на пересечение биграмм и слов ставим вероятность встретить второе после первого
    # вероятность встретить слова после биграммы
    matrix_news[bigram2id_news[bigram], word2id_news[word3]] =  (trigrams_news[trigram]/
                                                                     bigrams_news[bigram])

In [ ]:
# Для генерации нам понадобится функция np.random.choice,
# которая выбирает случайный объект из заданных.
# Ещё в неё можно подать вероятность каждого объекта
# и она будет доставать по ним (не только максимальный по вероятности)

def generate(matrix, id2bigram, id2word, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_id = bigram2id[start]
    for i in range(n):
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_id].toarray()[0])

# просто выбирать наиболее вероятное продолжение не получится
# можете попробовать раскоментировать следующую строчку и посмотреть что получается
# в современных языковых моделях есть специальный параметр, который
# позволяет регулировать разнообразность/случайность генерации
# он называется температура, чем выше температура тем ближе будет к argmax
# чем меньше температура тем ближе к полностью рандомной генерации
        #chosen = matrix[current_id].toarray().argmax()

        text.append(id2word[chosen])

        if id2word[chosen] == '<end>':
            current_id = bigram2id[start]
        else:
            current_bigram = id2bigram[current_id]
            second_word = current_bigram.split()[1]
            next_text = second_word + ' ' + id2word[chosen]
            current_id = bigram2id[next_text]

    return ' '.join(text)

In [ ]:
for i in range(5):
    text = generate(matrix_news, id2bigram_news,
               id2word_news, bigram2id_news, n=100, start='<start> <start>').replace('<end>', '\n')
    print(i+1)
    print(text)

1
по данным интерфакса видеозапись была сделанная грузинскими спецслужбами 
 по словам савчука штатная численность отдела составит 50 процентов голосов избирателей принявших участие в референдуме граждан рф поскольку никаких специальных мер по оказанию помощи палестинским беженцам unrwa в лагере беженцев айн эль-хельва в пригороде лос-анджелеса куда они прибыли на северный кавказ в плацдарм священной войны газават 
 в общем-то мы этим и приступили к обыскам на даче генпрокурора в течение часа летать кругами чтобы выработать общую позицию со странами снг более года 
 нам надо будет за пределами афганистана российская сторона утверждает что причины гибели военнослужащих и членов экипажа 
 похоронен
2
вместе с руководством кпрф 
 по имеющимся сведениям патентообладатели все-таки не включен в общефедеральный список сталинского блока житель нарвы юрий мишин баллотируется по списку лдпр 
 иосиро мори 
 устаналиваются тарифные квоты на добычу нефти на заводы внутри страны 
 в связи с эти през

Считаем перплексию:

In [ ]:
# Мы уже видели что произведение вероятностей можно заменить на экспоненту суммы логарифмов
# С возведением в степень тоже есть удобное правило - log(x^y) = y * log(x)
# можно заменить вот такую функцию (она ожидает вероятность)
# def perplexity(p, N):
#     return p**(-1/N)


# на вот такую (результат должен совпадать)
# функция ожидает логарифм вероятности

def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [ ]:
def compute_join_proba_markov_assumption(text, word_counts, bigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>'],2):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)

    return prob, len(tokens)

In [ ]:
p = []
for sent in test_news:
    text = ' '.join(sent)
    p.append(perplexity(*compute_join_proba_markov_assumption(text, unigrams_news, bigrams_news)))
print(np.mean(p))

20610.830649491192


## Задание № 2* (2 балла).

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

**1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?**

Несловарные (неизвестные) слова принято называть out of vocabulary (OOV) words. Процентное содержание таких слов в тексте (по сравнению с известным нам закрытым словарём) называется OOV rate.

Один из способов решить проблему с неизвестными словами заключается в том, чтобы добавить в наш закрытый словарь псевдо-слова, которые обозначаются как <UNK>. Таким образом создатся система с открытым словарём.

Если у нас нет заранее собранного закрытого словаря, мы можем собрать его, заменяя при этом особенно редкие слова на <UNK>.

И в том, и в другом случае полученный открытый словарь мы в дальнейшем используем для обучения модели. В процессе обучения мы относимся к <UNK> как к обычному слову и точно так же считаем вероятность встретить его, как для любого другого слова.

**2. Что такое сглаживание (smoothing)?**

Иногда известное нам слово встречается в редком для себя контексте (например, после слова, с которым оно не встречалось в обучающей выборке). Соответственно, языковая модель скорее всего присвоит нулевую вероятность такому событию. Чтобы такого не происходило, нужно уменьшить значения вероятности от некоторых более частотных событий и за счёт этого немного повысить вероятность более редких. Так и происходит сглаживание.